# A autoencoder for mnist dataset

## import module

In [8]:
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt

## Modify the class "Mode" called "MyAE"

In [9]:
# make subclass of Model for autoecndoer
class MyAE(tf.keras.Model):
    def __init__(self):
        super(MyAE, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),  # 26, 26, 16
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),  # 24, 24, 32
            tf.keras.layers.MaxPool2D(strides=3),  # 8, 8, 32
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),  # 6, 6, 32
            tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'),  # 4, 4, 16
            tf.keras.layers.GlobalMaxPool2D(),  # 16, 1
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Reshape((4, 4, 1), input_shape=(16,)),  # 4, 4, 1
            tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, activation='relu'),  # 6, 6, 16
            tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, activation='relu'),  # 8, 8, 32
            tf.keras.layers.UpSampling2D(size=(3, 3)),  # 24, 24, 32
            tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, activation='relu'),  # 26, 26, 16
            tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, activation='relu'),  # 28, 28, 1
        ])
    def call(self, inputs):
        latent_feature = self.encoder(inputs)
        return self.decoder(latent_feature)

## modefy you parameter at there

In [10]:
# para
BATCH_SIZE = 100
EPOCHS = 2000

## load data and create dataset

In [11]:
# load mnist images
(train_images_raw, _), (test_images_raw, _) = tf.keras.datasets.mnist.load_data()

# reshape to 4d data, and normalizing the value to the range of [0., 1.]
train_images = train_images_raw.reshape(train_images_raw.shape + (1,)).astype('float32') /255.
test_images = test_images_raw.reshape(test_images_raw.shape + (1,)).astype('float32') /255.

# Binarization
train_images[train_images >= .5] = 1.
train_images[train_images < .5] = 0.
test_images[test_images >= .5] = 1.
test_images[test_images < .5] = 0.

# create dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_images)).shuffle(train_images.shape[0]).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_images)).shuffle(test_images.shape[0]).batch(BATCH_SIZE)

## create my predict image

In [12]:
my_images = train_images[0:1]
my_show = my_images.reshape(my_images.shape[1:3]) * 255
my_show = Image.fromarray(my_show)
# plt.imshow(my_show)

## run the model

In [13]:
model = MyAE()
model.compile(optimizer='adam', loss='binary_crossentropy')
his = model.fit(train_dataset, epochs=EPOCHS, verbose=1, validation_data=test_dataset)

Epoch 1/1000
600/600 [==============================] - 4s 7ms/step - loss: 0.2466 - val_loss: 0.0000e+00 - loss -  - 3s 7ms/step - loss:  - 3s 7ms/step - loss: 0. - 3s - 3s 7ms/step - loss: 0.25 - 3s 7ms/step - loss: 0.24 - 3s 7ms/step - loss: 0. - 3s 7ms/step - loss: 0. 
Epoch 2/1000
255/600 [===========>..................] - ETA: 3s - loss: 0.2496

KeyboardInterrupt: 

## test the ae's output, not latent feature

In [ ]:
pre_images = model.predict(my_images)
pre_images = pre_images.reshape(pre_images.shape[1:3])

pre_images[pre_images >= .5] = 1.
pre_images[pre_images < .5] = 0.
pre_images = pre_images * 255
pre_images = Image.fromarray(pre_images)
# plt.imshow(pre_images)

In [ ]:
plt.plot(his.history['loss'])
plt.plot(his.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('ae_mnist_complicate.png')